In [93]:
import os
import numpy as np
import rasterio
import cv2
import matplotlib.pyplot as plt
from glob import glob
from datetime import datetime
import seaborn as sns
import pandas as pd

In [94]:
# Définir les répertoires d'entrée et de sortie
input_dir = '../data/raw/landsat'
cropped_dir = '../data/raw/cropped_straightened'
ndvi_output_dir = '../ndvi/ndvi_cropped_straightened_images'
os.makedirs(cropped_dir, exist_ok=True)
os.makedirs(ndvi_output_dir, exist_ok=True)

In [95]:
# Définition des classes NDVI
ndvi_classes = {
    -1: 'Eau',
    0: 'Sols nus/urbain',
    1: 'Végétation très faible',
    2: 'Végétation faible',
    3: 'Végétation modérée',
    4: 'Végétation dense',
    5: 'Forêt tropicale'
}

ndvi_bins = [-np.inf, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, np.inf]
ndvi_labels = list(ndvi_classes.values())

In [96]:
# ================================
# 1. Chargement des bandes B3 et B4
# ================================
def load_bands(red_band_path, nir_band_path):
    """ Charge les bandes Rouge (B3) et NIR (B4) et vérifie leur dimension """
    try:
        with rasterio.open(red_band_path) as red_src:
            red = red_src.read(1).astype('float64')
            red_meta = red_src.meta
    except Exception as e:
        print(f"Erreur lors du chargement de l'image Rouge (B3) : {red_band_path}")
        print(f"Exception : {e}")
        return None, None, None

    try:
        with rasterio.open(nir_band_path) as nir_src:
            nir = nir_src.read(1).astype('float64')
    except Exception as e:
        print(f"Erreur lors du chargement de l'image NIR (B4) : {nir_band_path}")
        print(f"Exception : {e}")
        return None, None, None

    return red, nir, red_meta

In [97]:
# ================================
# 2. Cropping des images
# ================================
def crop_image(image, date_str, band_name):
    """ Cropper l'image pour enlever les zones inutiles """
    if image is None:
        print(f"Image à croper est None pour {band_name} à la date {date_str}.")
        return None
    
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    _, thresholded = cv2.threshold(norm_image, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"Aucun contour détecté pour le cropping de {band_name} à la date {date_str}.")
        return None
    
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    cropped_image = image[y:y+h, x:x+w]
    
    return cropped_image

In [98]:
# ================================
# 3. Redressement des images
# ================================
def detect_and_straighten(image, date_str, band_name):
    """ Détecte les coins et redresse l'image en utilisant OpenCV """
    if image is None:
        print(f"Image à redresser est None pour {band_name} à la date {date_str}.")
        return None
    
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    _, thresholded = cv2.threshold(norm_image, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"Aucun contour détecté pour le redressement de {band_name} à la date {date_str}.")
        return None
    
    largest_contour = max(contours, key=cv2.contourArea)
    
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    if len(approx) >= 4:
        pts = approx.reshape(-1, 2)
        rect = order_points(pts)
        
        (tl, tr, br, bl) = rect
        widthA = np.linalg.norm(br - bl)
        widthB = np.linalg.norm(tr - tl)
        maxWidth = max(int(widthA), int(widthB))
        
        heightA = np.linalg.norm(tr - br)
        heightB = np.linalg.norm(tl - bl)
        maxHeight = max(int(heightA), int(heightB))
        
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        
        M = cv2.getPerspectiveTransform(rect, dst)
        straightened_image = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        return straightened_image
    else:
        print(f"Les coins n'ont pas été correctement détectés pour {band_name} à la date {date_str}.")
        return None

In [99]:
# ================================
# 4. Sauvegarde des images croppées et redressées
# ================================
def save_cropped_straightened(image, date_str, band_name):
    """ 
    Sauvegarde l'image croppée et redressée dans le dossier cropped_straightened 
    - Format TIF (sans compression) pour compatibilité
    - Format PNG pour ouverture facile sur Windows
    """
    save_path = os.path.join(cropped_dir, f"{date_str}_{band_name}_cropped_straightened.TIF")
    with rasterio.open(
        save_path,
        'w',
        driver='GTiff',
        height=image.shape[0],
        width=image.shape[1],
        count=1,
        dtype='float32',
        compress='none'
    ) as dst:
        dst.write(image, 1)
    
    # Sauvegarde en PNG pour ouverture facile sur Windows
    png_save_path = save_path.replace('.TIF', '.png')
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    cv2.imwrite(png_save_path, norm_image)
    
    print(f"Image croppée et redressée sauvegardée en TIF : {save_path}")
    print(f"Image croppée et redressée sauvegardée en PNG : {png_save_path}")

In [100]:
# ================================
# 5. Calcul et classification du NDVI
# ================================
def calculate_ndvi(red, nir):
    """ Calcul du NDVI """
    ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))
    return ndvi

def classify_ndvi(ndvi):
    """ Classification du NDVI en classes définies """
    ndvi_classified = np.digitize(ndvi, ndvi_bins) - 1
    return ndvi_classified

def calculate_and_save_ndvi(red, nir, date_str):
    """
    Calculer le NDVI sur les images croppées et redressées,
    puis sauvegarder le résultat.
    """
    # Vérification des images
    if red is None or nir is None:
        print(f"Impossible de calculer le NDVI pour la date {date_str} car une des images est None.")
        return None

    # Vérification des dimensions
    if red.shape != nir.shape:
        print(f"Les dimensions des images ne correspondent pas pour la date {date_str}. Recadrage en cours...")
        red, nir = crop_to_same_size(red, nir)
        
        # Double vérification après recadrage
        if red.shape != nir.shape:
            print(f"Échec du recadrage pour la date {date_str}. Dimensions Red: {red.shape}, NIR: {nir.shape}")
            return None
    
    # Calcul du NDVI
    ndvi = calculate_ndvi(red, nir)
    
    # Sauvegarde du NDVI
    save_path = os.path.join(ndvi_output_dir, f"{date_str}_ndvi.TIF")
    save_image(ndvi, save_path)
    
    return ndvi

def save_image(image, save_path):
    """ 
    Sauvegarde l'image NDVI au format TIF (sans compression) et PNG pour compatibilité Windows.
    """
    # Sauvegarde en TIF (sans compression pour compatibilité Windows)
    with rasterio.open(
        save_path,
        'w',
        driver='GTiff',
        height=image.shape[0],
        width=image.shape[1],
        count=1,
        dtype='float32',
        compress='none'
    ) as dst:
        dst.write(image, 1)
    
    # Sauvegarde en PNG pour ouverture facile sur Windows
    png_save_path = save_path.replace('.TIF', '.png')
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    cv2.imwrite(png_save_path, norm_image)
    
    print(f"Image NDVI sauvegardée en TIF (sans compression) : {save_path}")
    print(f"Image NDVI sauvegardée en PNG : {png_save_path}")

In [101]:
# ================================
# 6. Analyse temporelle complète
# ================================
def temporal_analysis(ndvi_series):
    """
    Réalisation de l'analyse temporelle sur les NDVI calculés :
    - Cartes temporelles
    - Carte de différence entre deux dates
    - Heatmap d'évolution des classes NDVI
    - Graphique de transition des classes NDVI
    - Analyse de tendance par classe NDVI
    """
    dates = sorted(ndvi_series.keys())
    
    # Vérification des dimensions et recadrage pour alignement parfait
    print("Vérification et recadrage des NDVI pour alignement parfait...")
    min_rows = min(ndvi.shape[0] for ndvi in ndvi_series.values())
    min_cols = min(ndvi.shape[1] for ndvi in ndvi_series.values())
    
    # Recadrer tous les NDVI à la même taille
    for date in dates:
        original_shape = ndvi_series[date].shape
        ndvi_series[date] = ndvi_series[date][:min_rows, :min_cols]
        print(f"NDVI pour la date {date.strftime('%Y-%m-%d')} recadré de {original_shape} à {ndvi_series[date].shape}")
    
    # 1. Cartes temporelles du NDVI
    for date, ndvi in ndvi_series.items():
        plt.figure(figsize=(10, 8))
        plt.title(f'NDVI - {date.strftime("%Y-%m-%d")}')
        plt.imshow(ndvi, cmap='RdYlGn')
        plt.colorbar(label='Valeur NDVI')
        plt.savefig(os.path.join(ndvi_output_dir, f"ndvi_{date.strftime('%Y-%m-%d')}.png"))
        plt.close()

    # 2. Carte de différence NDVI entre deux dates consécutives
    for i in range(len(dates) - 1):
        date1, date2 = dates[i], dates[i + 1]
        
        # Calcul de la différence en vérifiant les dimensions
        diff = ndvi_series[date2] - ndvi_series[date1]
        
        plt.figure(figsize=(10, 8))
        plt.title(f'Différence NDVI - {date2.strftime("%Y-%m-%d")} vs {date1.strftime("%Y-%m-%d")}')
        plt.imshow(diff, cmap='bwr', vmin=-1, vmax=1)
        plt.colorbar(label='Différence NDVI')
        plt.savefig(os.path.join(ndvi_output_dir, f"ndvi_diff_{date2.strftime('%Y-%m-%d')}_vs_{date1.strftime('%Y-%m-%d')}.png"))
        plt.close()

    print("Analyse temporelle complète terminée et visualisations sauvegardées.")

In [102]:
# ================================
# 7. Processus complet de traitement des images et calcul du NDVI
# ================================
def process_images():
    """
    Processus complet de traitement des images :
    - Chargement des bandes B3 (Rouge) et B4 (NIR)
    - Cropping pour enlever les zones en dehors de l'image satellite
    - Redressement pour aligner l'image dans un carré parfait
    - Sauvegarde des images croppées et redressées dans '../data/raw/cropped_straightened'
    - Calcul du NDVI sur les images croppées et redressées
    - Sauvegarde du NDVI dans '../ndvi/ndvi_cropped_straightened_images'
    - Retourne un dictionnaire des NDVI par date pour l'analyse temporelle
    """
    ndvi_series = {}
    
    # Parcours des dossiers par date
    for date_dir in sorted(glob(os.path.join(input_dir, '*'))):
        date_str = os.path.basename(date_dir)
        try:
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            continue
        
        # Rechercher les images B3 et B4
        b3_files = glob(os.path.join(date_dir, '*_B3.TIF'))
        b4_files = glob(os.path.join(date_dir, '*_B4.TIF'))
        
        if not b3_files or not b4_files:
            print(f"Aucune image B3 ou B4 trouvée pour la date : {date_str}")
            continue
        
        b3_path = b3_files[0]
        b4_path = b4_files[0]
        
        print(f"\n--- Traitement des images pour la date : {date_str} ---")
        
        # Charger les bandes
        red, nir, meta_data = load_bands(b3_path, b4_path)
        
        # Cropping des images
        print("--- Cropping des images ---")
        red_cropped = crop_image(red, date_str, 'B3_Rouge')
        nir_cropped = crop_image(nir, date_str, 'B4_NIR')
        
        # Redressement des images
        print("--- Redressement des images ---")
        red_straightened = detect_and_straighten(red_cropped, date_str, 'B3_Rouge')
        nir_straightened = detect_and_straighten(nir_cropped, date_str, 'B4_NIR')
        
        # Sauvegarde des images croppées et redressées
        print("--- Sauvegarde des images croppées et redressées ---")
        save_cropped_straightened(red_straightened, date_str, 'B3_Rouge')
        save_cropped_straightened(nir_straightened, date_str, 'B4_NIR')
        
        # Calcul du NDVI
        print("--- Calcul du NDVI ---")
        ndvi = calculate_and_save_ndvi(red_straightened, nir_straightened, date_str)
        ndvi_series[date_obj] = ndvi

    return ndvi_series

In [103]:
# ================================
# 8. Fonction principale
# ================================
if __name__ == "__main__":
    """
    Fonction principale :
    - Lance le processus complet de traitement des images
    - Effectue l'analyse temporelle sur les NDVI calculés
    """
    print("\n===============================")
    print("DÉMARRAGE DU TRAITEMENT COMPLET")
    print("===============================\n")

    # Processus complet de traitement des images et calcul du NDVI
    ndvi_series = process_images()

    # Analyse temporelle complète
    print("\n===============================")
    print("DÉMARRAGE DE L'ANALYSE TEMPORELLE")
    print("===============================\n")
    temporal_analysis(ndvi_series)
    
    print("\n===============================")
    print("TRAITEMENT COMPLET TERMINÉ")
    print("===============================\n")


DÉMARRAGE DU TRAITEMENT COMPLET


--- Traitement des images pour la date : 1998-02-16 ---
--- Cropping des images ---
--- Redressement des images ---
--- Sauvegarde des images croppées et redressées ---
Image croppée et redressée sauvegardée en TIF : ../data/raw/cropped_straightened\1998-02-16_B3_Rouge_cropped_straightened.TIF
Image croppée et redressée sauvegardée en PNG : ../data/raw/cropped_straightened\1998-02-16_B3_Rouge_cropped_straightened.png
Image croppée et redressée sauvegardée en TIF : ../data/raw/cropped_straightened\1998-02-16_B4_NIR_cropped_straightened.TIF
Image croppée et redressée sauvegardée en PNG : ../data/raw/cropped_straightened\1998-02-16_B4_NIR_cropped_straightened.png
--- Calcul du NDVI ---
Les dimensions des images ne correspondent pas pour la date 1998-02-16. Recadrage en cours...


C:\Users\billo.bah\AppData\Local\Temp\ipykernel_15652\207070673.py:6: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))


Image NDVI sauvegardée en TIF (sans compression) : ../ndvi/ndvi_cropped_straightened_images\1998-02-16_ndvi.TIF
Image NDVI sauvegardée en PNG : ../ndvi/ndvi_cropped_straightened_images\1998-02-16_ndvi.png

--- Traitement des images pour la date : 1998-03-04 ---
--- Cropping des images ---
--- Redressement des images ---
--- Sauvegarde des images croppées et redressées ---
Image croppée et redressée sauvegardée en TIF : ../data/raw/cropped_straightened\1998-03-04_B3_Rouge_cropped_straightened.TIF
Image croppée et redressée sauvegardée en PNG : ../data/raw/cropped_straightened\1998-03-04_B3_Rouge_cropped_straightened.png
Image croppée et redressée sauvegardée en TIF : ../data/raw/cropped_straightened\1998-03-04_B4_NIR_cropped_straightened.TIF
Image croppée et redressée sauvegardée en PNG : ../data/raw/cropped_straightened\1998-03-04_B4_NIR_cropped_straightened.png
--- Calcul du NDVI ---
Image NDVI sauvegardée en TIF (sans compression) : ../ndvi/ndvi_cropped_straightened_images\1998-03-04